In [12]:
import sys, os, time
import numpy as np
from datetime import datetime
from joblib import Parallel, delayed
import logging

parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from estimators.statistical_descriptor import Nagler_WS
# from plot.figure_roc import ROC_plot
from utils.dataset_management import parse_pipeline
from utils.dataset_load import shuffle_data, DatasetLoader
from utils.fold_management import FoldManagement
from utils.label_management import LabelManagement
from utils.figures import plot_boxplots, plot_roc_curves
from utils.files_management import (
    load_yaml,
    dump_pkl,
    init_logger,
    report_prediction,
    report_metric_from_log,
    set_folder,
    logger_dataset,
    logger_fold,
    save_metrics
)

def fit_predict_fold(pipeline, X_train_k, y_train_k, X_test_k, y_test_k, log_model, label_encoder, kfold, pipeline_name, save_dir):
    pipeline_id = f"{pipeline_name}_kfold_{kfold}"
    try:
        start_time = time.time()
        pipeline.fit(X_train_k, y_train_k)
        training_time = time.time() - start_time

        start_time = time.time()
        y_prob = pipeline.predict_proba(X_test_k)
        prediction_time = time.time() - start_time

        log_model, fold_metric = report_prediction(log_model, y_test_k, y_prob, label_encoder, kfold)

        fold_metric["training_time"] = training_time
        fold_metric["prediction_time"] = prediction_time

        dump_pkl(pipeline, os.path.join(save_dir, f"{pipeline_name}_fold{kfold}.pkl"))

        return fold_metric, y_prob, y_test_k
    except Exception as e:
        log_model.error(f"Pipeline {pipeline_id} failed")
        log_model.error(e)
        return None, None, None

def predict_dataset(
    x,
    targets,
    fold_groups,
    output_dir,
    pipeline_params,
    label_encoder,
    log_results,
    save=True
):
    y_est_save, metrics = {}, {}

    for count, pipeline_name in enumerate(pipeline_params["pipeline_names"]):
        save_dir = os.path.join(output_dir, f"models/{pipeline_name}/")
        log_model, _ = init_logger(save_dir, f"{pipeline_name}_results")

        log_model.info(f"================== Fitting model {pipeline_name} ==================")

        y_est_save[pipeline_name] = {"y_true": [], "y_est": []}
        fold_metrics = []

        def fit_predict_fold_wrap(fold, train_index, test_index):
            X_train_k, y_train_k = x[train_index], targets[train_index]
            X_test_k, y_test_k = x[test_index], targets[test_index]

            return fit_predict_fold(
                parse_pipeline(pipeline_params, count),
                X_train_k, y_train_k,
                X_test_k, y_test_k,
                log_model,
                label_encoder,
                fold,
                pipeline_name,
                save_dir
            )

        results = Parallel(n_jobs=7)(
            delayed(fit_predict_fold_wrap)(kfold, train_index, test_index)
            for kfold, (train_index, test_index) in enumerate(fold_groups)
        )

        for (fold_metric, y_prob, y_test_k) in results:
            if fold_metric is not None:
                fold_metrics.append(fold_metric)
                y_est_save[pipeline_name]["y_est"].extend(y_prob)
                y_est_save[pipeline_name]["y_true"].extend(y_test_k)
        
        log_model = save_metrics(log_model, fold_metrics, pipeline_name)

        if save:
            dump_pkl(fold_metrics, os.path.join(save_dir, "metrics.pkl"))
        metrics[pipeline_name] = fold_metrics

    results_dir = os.path.join(output_dir, "results/plots/")
    plot_boxplots(metrics, save_dir=results_dir)
    plot_roc_curves(metrics, save_dir=results_dir)
    log_results = report_metric_from_log(log_results, metrics, pipeline_params["metrics_to_report"])

    return y_est_save

In [7]:
param_path = "../parameter/config_pipeline.yml"
pipeline_params = load_yaml(param_path)

try:
    data_path = "../data/dataset/dataset_AD_08200821_14Mas3Top3Phy_W15_corrected_V2.h5"
    out_dir = "./tmp/"
    fold_method = pipeline_params["fold_method"]
    seed = pipeline_params["seed"]
    labeling_method = pipeline_params["labeling_method"]
    resampling_method = pipeline_params["resampling_method"]
    balance_data = pipeline_params["balance_data"]
    # orbit = pipeline_params["orbit"]
    request = pipeline_params["request"]
    shuffle_data = pipeline_params["shuffle_data"]
    channel_transformation = pipeline_params["channel_transformation"]
    BANDS_MAX = pipeline_params["BANDS_MAX"]

except KeyError as e:
    print("KeyError: %s undefined" % e)

dataset_loader = DatasetLoader(
    data_path,
    shuffle=shuffle_data,
    descrp=[
        "date",
        "massif",
        "acquisition",
        "elevation",
        "slope",
        "orientation",
        "tmin",
        "hsnow",
        "tel"
    ],
    print_info=True,
    seed=seed
)

x, y = dataset_loader.request_data(request)

Request: (date.dt.month == 3 and date.dt.day == 1) and ((elevation > 1000) and (elevation < 2000)) with 9125 samples
(9125, 15, 15, 9)


In [18]:
out_dir = set_folder(out_dir, pipeline_params)
log_dataset, _ = init_logger(out_dir, "dataset_info")
log_results, _ = init_logger(out_dir + "results", "results")

labels_manager = LabelManagement(method=labeling_method)

targets = labels_manager.transform(y)
label_encoder = labels_manager.get_encoder()

fold_manager = FoldManagement(targets=targets,
                                method=fold_method,
                                resampling_method=resampling_method, 
                                shuffle=shuffle_data, 
                                random_state=seed,
                                balanced=balance_data,
                                train_aprox_size=0.8)

fold_groups = fold_manager.split(x, y)
log_dataset = logger_dataset(log_dataset, x, y, targets)
log_dataset = logger_fold(log_dataset, fold_groups, targets, y)

fold_groups

[([2053,
   39,
   3977,
   1057,
   4539,
   2750,
   6061,
   6314,
   3092,
   5612,
   7767,
   5531,
   8329,
   8462,
   2218,
   4768,
   3243,
   4760,
   3993,
   1967,
   4299,
   1138,
   3789,
   5278,
   4030,
   6530,
   1641,
   3333,
   7144,
   3039,
   1416,
   7159,
   5193,
   25,
   7422,
   87,
   1709,
   6463,
   1276,
   5848,
   1362,
   6057,
   837,
   7483,
   5097,
   1271,
   8820,
   566,
   4946,
   6475,
   8062,
   7071,
   6631,
   1929,
   5975,
   3153,
   895,
   1134,
   5667,
   6176,
   1518,
   7521,
   8936,
   6224,
   7072,
   6696,
   6762,
   160,
   2024,
   2691,
   3583,
   3741,
   3162,
   6385,
   2781,
   6716,
   3272,
   2713,
   1861,
   8659,
   4976,
   5880,
   398,
   4892,
   3283,
   1061,
   1788,
   7647,
   1037,
   3042,
   2942,
   8178,
   6506,
   5694,
   2766,
   5681,
   403,
   5301,
   4279,
   5001,
   8741,
   5568,
   7406,
   6476,
   6624,
   4569,
   5195,
   1534,
   6130,
   2919,
   137,
   1690,
   14

In [9]:

y_est_save = predict_dataset(x=x,
                                targets=targets,
                                fold_groups=fold_groups,
                                output_dir=out_dir,
                                pipeline_params=pipeline_params,
                                label_encoder=label_encoder,
                                log_results=log_results,
                                save=True)

print("================== End of the study ==================")

[Pipeline] ...... (step 1 of 3) Processing BandSelector, total=   0.0s
[Pipeline] ...... (step 1 of 3) Processing BandSelector, total=   0.0s
[Pipeline] ...... (step 1 of 3) Processing BandSelector, total=   0.0s
[Pipeline] ...... (step 1 of 3) Processing BandSelector, total=   0.0s
[Pipeline] ...... (step 1 of 3) Processing BandSelector, total=   0.0s
[Pipeline] ...... (step 1 of 3) Processing BandSelector, total=   0.0s
[Pipeline] ...... (step 1 of 3) Processing BandSelector, total=   0.0s


 77%|███████▋  | 700/906 [00:01<00:00, 636.05it/s]

[Pipeline] .......... (step 2 of 3) Processing Hist_SAR, total=   1.0s
[Pipeline] .......... (step 2 of 3) Processing Hist_SAR, total=   1.0s
[Pipeline]  (step 3 of 3) Processing KNeighborsClassifier, total=   0.0s
[Pipeline]  (step 3 of 3) Processing KNeighborsClassifier, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing Hist_SAR, total=   1.1s


100%|██████████| 870/870 [00:01<00:00, 691.68it/s]


[Pipeline] .......... (step 2 of 3) Processing Hist_SAR, total=   1.2s
[Pipeline]  (step 3 of 3) Processing KNeighborsClassifier, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing Hist_SAR, total=   1.3s
[Pipeline] .......... (step 2 of 3) Processing Hist_SAR, total=   1.3s
[Pipeline]  (step 3 of 3) Processing KNeighborsClassifier, total=   0.0s
[Pipeline]  (step 3 of 3) Processing KNeighborsClassifier, total=   0.0s
[Pipeline] .......... (step 2 of 3) Processing Hist_SAR, total=   1.4s


...

[Pipeline]  (step 3 of 3) Processing LogisticRegression, total=   0.0s


 93%|█████████▎| 1771/1914 [00:02<00:00, 702.72it/s]Pipeline LogisticR_direct_kfold_3 failed
axis 1 is out of bounds for array of dimension 1
 92%|█████████▏| 1753/1896 [00:02<00:00, 700.12it/s]Pipeline LogisticR_direct_kfold_1 failed
axis 1 is out of bounds for array of dimension 1
100%|██████████| 1862/1862 [00:02<00:00, 663.96it/s]
Pipeline LogisticR_direct_kfold_4 failed
axis 1 is out of bounds for array of dimension 1
 96%|█████████▋| 1843/1914 [00:02<00:00, 649.56it/s]/home/listic/miniconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_resul

[Pipeline]  (step 3 of 3) Processing LogisticRegression, total=   0.1s


100%|██████████| 1896/1896 [00:03<00:00, 626.79it/s]Pipeline LogisticR_direct_kfold_2 failed
axis 1 is out of bounds for array of dimension 1

100%|██████████| 1863/1863 [00:03<00:00, 605.67it/s]
Pipeline LogisticR_direct_kfold_6 failed
axis 1 is out of bounds for array of dimension 1
  7%|▋         | 126/1810 [00:00<00:02, 585.08it/s]Pipeline LogisticR_direct_kfold_5 failed
axis 1 is out of bounds for array of dimension 1
100%|██████████| 1901/1901 [00:02<00:00, 734.36it/s]
Pipeline LogisticR_direct_kfold_7 failed
axis 1 is out of bounds for array of dimension 1
100%|██████████| 1810/1810 [00:02<00:00, 741.04it/s]
Pipeline LogisticR_direct_kfold_8 failed
axis 1 is out of bounds for array of dimension 1


FileNotFoundError: [Errno 2] No such file or directory: './tmp/study_140624_11H40M51_crocus_combinationFold/study_140624_11H42M59_crocus_combinationFold/models/LogisticR_direct/metrics.pkl'